🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../../README.md#setup-instructions) to configure your environment and API keys.

# Building a Sentiment Analysis Pipeline with Custom Components

Welcome to this comprehensive tutorial on building a sentiment analysis pipeline using Haystack with custom components! This notebook demonstrates how to create a sophisticated system that fetches business reviews from the Yelp API and performs sentiment analysis using transformer models.

## What you'll learn

After completing this notebook, you will be able to:
- Build custom Haystack components that integrate with external APIs
- Implement sentiment analysis using transformer-based models
- Create robust data processing pipelines for real-world applications
- Handle API authentication and data extraction patterns
- Combine multiple processing steps in a reusable component
- Structure Haystack pipelines with custom business logic

## Key Concepts

Before we begin, let's understand some core concepts:
- **Custom Components**: Extend Haystack functionality with domain-specific logic
- **API Integration**: Fetch external data within pipeline workflows
- **Sentiment Analysis**: Classify text as positive, negative, or neutral using ML models
- **Component Composition**: Build complex pipelines from modular, reusable parts
- **Document Metadata**: Enrich documents with additional context and analysis results


Let's dive in and build our sentiment analysis pipeline step by step!

## 1 Load environment variables

We'll be working with the Yelp Reviews API https://rapidapi.com/beat-analytics-beat-analytics-default/api/yelp-business-reviews

Ensure to obtain a key and store it in your `.venv` file

In [5]:
## 1. Environment Setup

# We begin by loading environment variables (such as API keys) from a `.env` file.
# This keeps sensitive information secure and out of the codebase.

import requests
from dotenv import load_dotenv
import os 

# Load environment variables from .env file
load_dotenv(".env")

# Retrieve the RapidAPI key for Yelp Business Reviews API
RAPID_API_KEY = os.getenv("RAPID_API_KEY")

## 2. Sentiment Analysis Model Setup

We'll use Haystack's `TransformersTextRouter` component with a pre-trained sentiment analysis model:

- **Model**: `cardiffnlp/twitter-roberta-base-sentiment`
- **Architecture**: RoBERTa (Robustly optimized BERT approach)
- **Training Data**: Twitter posts (ideal for short-form social media text)
- **Output Classes**: 
  - LABEL_0: Negative sentiment
  - LABEL_1: Neutral sentiment  
  - LABEL_2: Positive sentiment

The `warm_up()` method loads the model into memory, ensuring fast inference when processing reviews.

In [6]:
## 2. Initialize Sentiment Analysis Model

# We'll use a pre-trained sentiment analysis model from Hugging Face
# This model is based on RoBERTa and trained on Twitter data
# It classifies text into three categories: negative, neutral, and positive

from haystack.components.routers import TransformersTextRouter

# Initialize the sentiment analysis router with a pre-trained model
text_router = TransformersTextRouter(model="cardiffnlp/twitter-roberta-base-sentiment")

# Warm up the model to load it into memory for faster inference
text_router.warm_up()

Device set to use mps


## 3. Anatomy of Our Custom Components

### Core Concepts

Following Haystack best practices, we'll build **three separate components** instead of one monolithic component:

1. **YelpReviewFetcher**: Fetches reviews from the API and creates Documents
2. **TransformersTextRouter**: Routes each review text based on sentiment classification (built-in Haystack component)
3. **SentimentDocumentEnricher**: Enriches documents with human-readable sentiment labels


### Pipeline Architecture

```
Input (API credentials + params)
    ↓
YelpReviewFetcher
    ↓
Documents (with rating, url)
    ↓
[Loop through each document]
    ↓
TransformersTextRouter (sentiment classification)
    ├─→ LABEL_0 (negative) → SentimentDocumentEnricher
    ├─→ LABEL_1 (neutral)  → SentimentDocumentEnricher  
    └─→ LABEL_2 (positive) → SentimentDocumentEnricher
           ↓
Output: Documents with {rating, url, sentiment}
```

### Component Details

**1. YelpReviewFetcher**
- **Input**: url, headers, querystring
- **Processing**: Makes API call, extracts review data
- **Output**: List[Document] with metadata (rating, url)

**2. TransformersTextRouter** (Haystack built-in)
- **Input**: text (string)
- **Processing**: Classifies text into sentiment labels
- **Output**: Routes to label-specific sockets (LABEL_0, LABEL_1, LABEL_2)

**3. SentimentDocumentEnricher**
- **Input**: text (from router), original document
- **Processing**: Maps label to human-readable sentiment
- **Output**: Document with added sentiment metadata

In [18]:
## 4. Implementing the Custom Components

# Import necessary classes from Haystack
from haystack import component, Document
from typing import Any, Dict, List

# ============================================================================
# Component 1: YelpReviewFetcher
# ============================================================================

@component
class YelpReviewFetcher:
    """
    Fetches business reviews from the Yelp API via RapidAPI and converts them
    into Haystack Document objects with metadata.
    
    This component handles API integration and data extraction, creating Documents
    that can be processed by downstream components.
    
    Input Sockets:
        - url (str): The Yelp API endpoint URL
        - headers (Dict): Request headers including API authentication
        - querystring (Dict): Query parameters for filtering/sorting reviews
    
    Output Sockets:
        - documents (List[Document]): Reviews as Document objects with rating and URL metadata
    """
    
    def __init__(self):
        """Initialize the YelpReviewFetcher component."""
        pass
    
    @component.output_types(documents=List[Document])
    def run(self, url: str, headers: Dict, querystring: Dict) -> Dict[str, List[Document]]:
        """
        Fetch reviews from the Yelp API and convert them to Documents.
        
        Args:
            url (str): Yelp API URL endpoint for business reviews
            headers (Dict): Request headers containing API authentication
            querystring (Dict): Query parameters (e.g., sortBy, limit)
        
        Returns:
            Dict[str, List[Document]]: Dictionary with 'documents' key containing
                                       list of review Documents with metadata
        """
        try:
            # Make the API request
            response = requests.get(url, headers=headers, params=querystring)
            
            # Parse JSON response
            results = response.json()
            
            # Create Document objects for each review
            documents = []
            for i, review_data in enumerate(results['reviews']):
                doc = Document(
                    content=review_data['text'],
                    meta={
                        "rating": review_data['rating'],
                        "url": review_data['url'],
                        "review_id": i
                    }
                )
                documents.append(doc)
            
            return {"documents": documents}
            
        except Exception as e:
            # Return empty list on error to prevent pipeline failure
            print(f"Error fetching reviews: {e}")
            return {"documents": []}


# ============================================================================
# Component 2: SentimentDocumentEnricher  
# ============================================================================

@component
class SentimentDocumentEnricher:
    """
    Enriches documents with sentiment labels based on TransformersTextRouter output.
    
    This component receives text that has been routed through a sentiment classifier
    and adds a human-readable sentiment label to the document metadata.
    
    Input Sockets:
        - text (str): The review text (routed from TransformersTextRouter)
        - documents (List[Document]): Original documents to enrich
        - label (str): The sentiment label from the router
    
    Output Sockets:
        - documents (List[Document]): Documents enriched with sentiment metadata
    """
    
    def __init__(self):
        """Initialize the SentimentDocumentEnricher component."""
        # Mapping from model labels to human-readable sentiment
        self.sentiment_mapping = {
            "LABEL_0": "negative",
            "LABEL_1": "neutral",
            "LABEL_2": "positive"
        }
    
    @component.output_types(documents=List[Document])
    def run(self, text: str, documents: List[Document], label: str) -> Dict[str, List[Document]]:
        """
        Enrich documents with sentiment labels.
        
        Args:
            text (str): The review text that was classified
            documents (List[Document]): Original documents from YelpReviewFetcher
            label (str): The classification label (LABEL_0, LABEL_1, or LABEL_2)
        
        Returns:
            Dict[str, List[Document]]: Dictionary with 'documents' key containing
                                       enriched documents with sentiment metadata
        """
        enriched_documents = []
        
        # Find matching document by text content
        for doc in documents:
            if doc.content == text:
                # Create new document with enriched metadata
                enriched_doc = Document(
                    content=doc.content,
                    meta={
                        **doc.meta,  # Keep existing metadata (rating, url)
                        "sentiment": self.sentiment_mapping.get(label, "unknown")
                    }
                )
                enriched_documents.append(enriched_doc)
        
        return {"documents": enriched_documents}

## 5. Stand-alone usage


We'll test the review fetcher with lowest-rated reviews to ensure it correctly extracts data from the API.

In [19]:
# --- Test YelpReviewFetcher Component ---

# Define the API endpoint for a specific business
url = "https://yelp-business-reviews.p.rapidapi.com/reviews/RJNAeNA-209sctUO0dmwuA"

# Query parameters to sort by lowest rated reviews
querystring = {"sortBy": "lowestRated"}

# Set up headers with API authentication
headers = {
    "x-rapidapi-key": RAPID_API_KEY,
    "x-rapidapi-host": "yelp-business-reviews.p.rapidapi.com"
}

# Instantiate and test the YelpReviewFetcher component
review_fetcher = YelpReviewFetcher()
fetcher_result = review_fetcher.run(url=url, headers=headers, querystring=querystring)

print("--- YelpReviewFetcher Results ---")
print(f"Fetched {len(fetcher_result['documents'])} reviews\n")

# Display first review
if fetcher_result['documents']:
    first_doc = fetcher_result['documents'][0]
    print(f"Sample Document:")
    print(f"  Content: {first_doc.content[:100]}...")
    print(f"  Metadata: {first_doc.meta}")

--- YelpReviewFetcher Results ---
Fetched 10 reviews

Sample Document:
  Content: Went in on a Tuesday for lunch. Wasn't overly busy,  cheese curds were great however.... Server didn...
  Metadata: {'rating': 1, 'url': 'https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=pkO8UmZLWRI5Qfx0lJVRaQ', 'review_id': 0}


This contains the rating and content. We will apply the `TransformersTextRouter` to the content.

In [20]:
### Test 2: TransformersTextRouter Component

# Test the sentiment router on a sample review
if fetcher_result['documents']:
    sample_text = fetcher_result['documents'][0].content
    
    print("\n--- TransformersTextRouter Test ---")
    print(f"Testing with text: {sample_text[:100]}...")
    
    # Run the router
    router_result = text_router.run(text=sample_text)
    
    print(f"\nRouter output: {router_result}")
    print(f"Detected label: {list(router_result.keys())[0]}")


--- TransformersTextRouter Test ---
Testing with text: Went in on a Tuesday for lunch. Wasn't overly busy,  cheese curds were great however.... Server didn...

Router output: {'LABEL_2': "Went in on a Tuesday for lunch. Wasn't overly busy,  cheese curds were great however.... Server didn't really check in on us and although the kids got their food the adults didn't. The server never thought of checking to see where the food was."}
Detected label: LABEL_2


## 6. Building the Sentiment Analysis Pipeline

Now let's integrate our custom components with the built-in `TransformersTextRouter` into a proper Haystack pipeline.

### Challenge: Routing Multiple Documents

The `TransformersTextRouter` processes **one text string at a time**, but our `YelpReviewFetcher` returns **multiple documents**. To handle this, we'll use a custom component to batch process (for production)

### Pipeline Architecture for Single Review

```
YelpReviewFetcher → documents
         ↓
[Select one document]
         ↓
TransformersTextRouter (text from document)
    ├─→ LABEL_0 → SentimentDocumentEnricher(negative)
    ├─→ LABEL_1 → SentimentDocumentEnricher(neutral)
    └─→ LABEL_2 → SentimentDocumentEnricher(positive)
         ↓
Output: Document with sentiment
```


In [15]:
@component
class BatchSentimentProcessor:
    """
    Processes multiple documents through sentiment analysis in batch.
    
    This component demonstrates how to wrap TransformersTextRouter functionality
    for batch processing within a pipeline.
    
    Input Sockets:
        - documents (List[Document]): Documents to analyze
    
    Output Sockets:
        - documents (List[Document]): Documents enriched with sentiment metadata
    """
    
    def __init__(self, model: str = "cardiffnlp/twitter-roberta-base-sentiment"):
        """
        Initialize the batch sentiment processor.
        
        Args:
            model: Hugging Face model for sentiment classification
        """
        self.router = TransformersTextRouter(model=model)
        self.sentiment_mapping = {
            "LABEL_0": "negative",
            "LABEL_1": "neutral",
            "LABEL_2": "positive"
        }
    
    def warm_up(self):
        """Warm up the underlying transformer model."""
        self.router.warm_up()
    
    @component.output_types(documents=List[Document])
    def run(self, documents: List[Document]) -> Dict[str, List[Document]]:
        """
        Process documents through sentiment analysis.
        
        Args:
            documents: List of documents to analyze
        
        Returns:
            Dictionary with 'documents' key containing enriched documents
        """
        enriched_docs = []
        
        for doc in documents:
            # Classify the document content
            router_result = self.router.run(text=doc.content)
            label = list(router_result.keys())[0]
            sentiment = self.sentiment_mapping.get(label, "unknown")
            
            # Create enriched document
            enriched_doc = Document(
                content=doc.content,
                meta={
                    **doc.meta,
                    "sentiment": sentiment
                }
            )
            enriched_docs.append(enriched_doc)
        
        return {"documents": enriched_docs}


# Initialize and warm up the batch processor
batch_processor = BatchSentimentProcessor()
batch_processor.warm_up()

print("BatchSentimentProcessor initialized and ready")

Device set to use mps


BatchSentimentProcessor initialized and ready


## 7. Building a Batching Sentiment Analysis Pipeline (Production Pattern)

For production use, we create a **batch processing component** that handles multiple documents efficiently. This demonstrates the recommended pattern for integrating `TransformersTextRouter` into larger workflows.

### Why Batch Processing?

The `TransformersTextRouter` is designed to classify **one text at a time** and route it to different pipeline branches based on the classification label. However, when processing multiple documents:
  
- **Pipeline Approach**: Wrap router in a batch component (shown below)
  - ✅ Can be serialized and deployed
  - ✅ Follows best practices
  - ✅ Scalable and maintainable

### BatchSentimentProcessor Component

This component demonstrates how to integrate `TransformersTextRouter` into a pipeline-friendly component that processes multiple documents:

In [21]:
review_fetcher = YelpReviewFetcher()
batch_processor = BatchSentimentProcessor()
# Build the sentiment analysis pipeline
sentiment_pipeline = Pipeline()

# Add components
sentiment_pipeline.add_component("review_fetcher", review_fetcher)
sentiment_pipeline.add_component("sentiment_processor", batch_processor)

# Connect components: review_fetcher outputs -> sentiment_processor inputs
sentiment_pipeline.connect("review_fetcher.documents", "sentiment_processor.documents")

print("Pipeline structure:")
print(sentiment_pipeline)

Pipeline structure:
🚅 Components
  - review_fetcher: YelpReviewFetcher
  - sentiment_processor: BatchSentimentProcessor
🛤️ Connections
  - review_fetcher.documents -> sentiment_processor.documents (List[Document])



### Understanding the Component Connections

The pipeline connects components through their input/output sockets:

**Component 1: YelpReviewFetcher**
- **Inputs**: url, headers, querystring (provided at pipeline run time)
- **Outputs**: `documents` socket → List[Document] with metadata {rating, url}

**Component 2: BatchSentimentProcessor**  
- **Inputs**: `documents` socket (connected to YelpReviewFetcher output)
- **Processing**: Uses TransformersTextRouter internally for each document
- **Outputs**: `documents` socket → List[Document] with metadata {rating, url, sentiment}

The connection is made with:
```python
pipeline.connect("review_fetcher.documents", "sentiment_processor.documents")
```

This means: "Take the `documents` output from `review_fetcher` and pass it as the `documents` input to `sentiment_processor`".

In [23]:
sentiment_pipeline.draw(path="./images/sentiment_pipeline.png")

![](./images/sentiment_pipeline.png)

The pipeline diagram shows the two-component architecture:
1. **review_fetcher** (YelpReviewFetcher): Fetches reviews from API
2. **sentiment_processor** (BatchSentimentProcessor): Analyzes sentiment using TransformersTextRouter

Documents flow from left to right, getting enriched with sentiment metadata.

In [24]:
# Prepare the input data for the pipeline
# The key 'review_fetcher' matches the component name we used
pipeline_input = {
    "review_fetcher": {
        "url": url,
        "headers": headers,
        "querystring": {"sortBy": "highestRated"}
    }
}

# Execute the pipeline
print("--- Running Sentiment Analysis Pipeline ---\n")
pipeline_result = sentiment_pipeline.run(pipeline_input)

# Extract the enriched documents from the pipeline output
enriched_reviews = pipeline_result['sentiment_processor']['documents']

# Display results
print(f"Processed {len(enriched_reviews)} reviews with sentiment analysis\n")
print("--- Sample Results ---")

for i, doc in enumerate(enriched_reviews[:3]):  # Show first 3
    print(f"\n--- Review {i+1} ---")
    print(f"Rating: {doc.meta['rating']}/5")
    print(f"Sentiment: {doc.meta['sentiment']}")
    print(f"Review: {doc.content[:150]}...")
    print(f"URL: {doc.meta['url']}")

--- Running Sentiment Analysis Pipeline ---



Device set to use mps
/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch8/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Processed 10 reviews with sentiment analysis

--- Sample Results ---

--- Review 1 ---
Rating: 5/5
Sentiment: positive
Review: Food 5/5
Service 4.5/5
Ambiance 4.5/5

A Wisconsin staple at this point, The Old Fashioned does pub food (and even some brunch) well. The cheese curds...
URL: https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=-AHOyDmPRahLrNXnXYS0PA

--- Review 2 ---
Rating: 5/5
Sentiment: positive
Review: Came to Old Fashion on a mission to finally try their famous cheese curds and a Spotted Cow on tap (because when in Wisconsin, right?). The cheese cur...
URL: https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=wwE2C7-nNV8v7aGwTso4Ug

--- Review 3 ---
Rating: 5/5
Sentiment: positive
Review: Very good and recommend for people who are visiting. We come from south Texas and looked up the reviews here and they were spot on. We got the chicken...
URL: https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=UXwgu5Ez3eAjQY10Hd10nw


---

## Summary and Next Steps

### What You've Accomplished

Congratulations! You've successfully built a modular sentiment analysis pipeline using Haystack's component architecture. You've learned:

✅ **Modular Component Design**: How to build focused, single-responsibility components  
✅ **Component Integration**: Connecting custom components with built-in Haystack components (TransformersTextRouter)  
✅ **API Integration**: Techniques for fetching external data within pipeline workflows  
✅ **Sentiment Analysis**: Applying transformer models for text classification  
✅ **Document Metadata**: Enriching documents with analysis results and contextual information  
✅ **Pipeline Construction**: Building, connecting, visualizing, and executing Haystack pipelines  
✅ **Batch Processing**: Handling multiple documents efficiently in production-ready patterns

### Key Takeaways

1. **Separation of Concerns**: Breaking functionality into discrete components improves reusability and testability
2. **Component Composition**: Haystack's power comes from connecting small, focused components
3. **Built-in Components**: Leverage existing Haystack components (like TransformersTextRouter) rather than reimplementing
4. **Type Annotations**: Explicit input/output types enable pipeline validation and clear contracts
5. **Batch vs. Stream**: Consider whether to process items individually or in batches based on your use case
6. **Metadata Enrichment**: Store analysis results in document metadata for downstream processing

### Extension Ideas

Want to take this further? Try these enhancements:

1. **Multi-Component Pipeline Extensions**  
   - Add a `DocumentWriter` to save enriched reviews to a document store
   - Include a `PromptBuilder` and `OpenAIGenerator` to generate sentiment summaries
   - Add a `ConditionalRouter` to route reviews by sentiment (positive/negative/neutral)
   - Use `DocumentJoiner` to merge reviews from multiple businesses

2. **Enhanced Analysis**  
   - Extract keywords using NER (Named Entity Recognition)
   - Compute aggregate sentiment scores per business
   - Compare sentiment vs. numeric ratings to detect discrepancies
   - Track sentiment trends over time
   - Identify topics using topic modeling

3. **Data Processing Improvements**  
   - Add `DocumentCleaner` to normalize review text
   - Implement `DocumentSplitter` for very long reviews
   - Create embeddings with `SentenceTransformersDocumentEmbedder` for semantic search
   - Filter spam or low-quality reviews

4. **Production Readiness**  
   - Add rate limiting for API calls (use `time.sleep()` or a rate limiter component)
   - Implement caching to avoid redundant API requests
   - Add comprehensive error handling and logging
   - Create unit tests for each custom component
   - Add retry logic for failed API calls
   - Monitor component performance metrics

5. **Different Data Sources**  
   - Adapt YelpReviewFetcher for other review platforms (Amazon, Google Reviews, TripAdvisor)
   - Process social media posts (Twitter/X, Reddit, Instagram)
   - Analyze customer support tickets
   - Monitor product feedback from multiple sources

6. **Advanced Sentiment Analysis**
   - Use more sophisticated models (BERT, RoBERTa-large, domain-specific models)
   - Implement aspect-based sentiment analysis (sentiment per product feature)
   - Add emotion detection (happy, sad, angry, etc.)
   - Calculate sentiment intensity/confidence scores



